In [ ]:
import h5py
import keras
import matplotlib.pyplot as plt
import numpy as np
from os import access
import pandas as pd
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive/DB/Review/reviews_processed_final/"

review1.csv


In [ ]:
l = os.listdir('/content/drive/My Drive/DB/Review/reviews_processed_final/')
print(l)

['review1.csv']


In [ ]:
dir = '/content/drive/My Drive/DB/Review/reviews_processed_final/review1.csv'
df = pd.read_csv(dir, sep=',')
df.dropna(inplace=True)
df = df.loc[:,['basemodel_text', 'label']]
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 999987 entries, 0 to 999999
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   basemodel_text  999987 non-null  object
 1   label           999987 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 22.9+ MB


,basemodel_text,label
0,apparently pride osteria rough summer evidence...,1
1,store pretty good not great walmart prefer mil...,1
2,call wvm recommendation couple friend used pas...,1
3,ive stay many marriott renaissance marriott hu...,0
4,food alway great service manager well staff su...,1


In [ ]:
df['label'].value_counts()/df['label'].count()

1    0.664248
0    0.335752
Name: label, dtype: float64

In [ ]:
df['label'].value_counts()

1    664239
0    335748
Name: label, dtype: int64

In [ ]:
#Decreasing the number of positive reviews to 335748 to maintain class balance
positive_reviews = df[df['label']==1].head(335748)
negative_rewviews = df[df['label']==0]
df_main = pd.concat([positive_reviews, negative_rewviews], ignore_index=True)
df_main.info()
df_main.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671496 entries, 0 to 671495
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   basemodel_text  671496 non-null  object
 1   label           671496 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 10.2+ MB


,basemodel_text,label
0,apparently pride osteria rough summer evidence...,1
1,store pretty good not great walmart prefer mil...,1
2,call wvm recommendation couple friend used pas...,1
3,food alway great service manager well staff su...,1
4,probably one better breakfast sandwich ive eve...,1


In [ ]:
df_main['label'].value_counts()/df_main['label'].count()  #classes are balanced

1    0.5
0    0.5
Name: label, dtype: float64

In [ ]:
text_list  = df_main.loc[:,'basemodel_text'].values
y = df_main.loc[:,'label'].values

#applying count vectorizer on text
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=250, lowercase=True)
vectorizer.fit(text_list)
vector = vectorizer.transform(text_list)

print("Vocabulary : ",len(vectorizer.vocabulary_))
print("Final doc : ",type(vector.toarray()))

X = vector.toarray()

Vocabulary :  250
Final doc :  <class 'numpy.ndarray'>


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
print(len(X_train))
print(len(X_test))

537196
134300


In [ ]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
import time
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score


start = time.time()
model1 = LogisticRegression(random_state=0)
model1.fit(X_train, y_train)
y_train_pred1 = model1.predict(X_train)
end = time.time()

y_pred1 = model1.predict(X_test)
print("Train Accuracy : ",accuracy_score(y_train, y_train_pred1))
print("Test Accuracy : ",accuracy_score(y_test, y_pred1))
print("F1 score : ",f1_score(y_test, y_pred1))

Train Accuracy :  0.8271952881257493
Test Accuracy :  0.8280044676098287
F1 score :  0.8274661826547456


In [ ]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
import time
from sklearn.metrics import accuracy_score


start = time.time()
model_dt = DecisionTreeClassifier(criterion='entropy',random_state=0)
model_dt.fit(X_train, y_train)
y_train_pred2 = model_dt.predict(X_train)

end = time.time()


y_pred2 = model_dt.predict(X_test)
print("Train Accuracy ",accuracy_score(y_train, y_train_pred2))
print("Test Accuracy ",accuracy_score(y_test, y_pred2))
print("F1 score : ",f1_score(y_test, y_pred2))

Train Accuracy  0.9988477203851108
Test Accuracy  0.7331198808637379
F1 score :  0.7332033169075941
